# Load Processed Data into Vector Database

## Configuration

In [1]:
class MyConfig:
    pass
MY_CONFIG = MyConfig()

MY_CONFIG.INPUT_DATA_DIR = 'data/granite-docs/output_final/'
# MY_CONFIG.INPUT_DATA_REMOTE = "https://github.com/sujee/data-prep-kit-examples/blob/main/requirements.txt"

MY_CONFIG.EMBEDDING_MODEL = "BAAI/bge-small-en-v1.5"
MY_CONFIG.EMBEDDING_LENGTH = 384

MY_CONFIG.DB_URI = './rag_demo_dataprepkit_1.db'
MY_CONFIG.COLLECTION_NAME = 'dataprepkit_granite_docs'

## Step-: Load Parquet Data

Load all  `.parquet` files in the given dir

In [2]:
import pandas as pd
import glob

print ('Loading data from : ', MY_CONFIG.INPUT_DATA_DIR)

# Get a list of all Parquet files in the directory
parquet_files = glob.glob(f'{MY_CONFIG.INPUT_DATA_DIR}/*.parquet')
print ("Number of parquet files to read : ", len(parquet_files))
print ()

# Create an empty list to store the DataFrames
dfs = []

# Loop through each Parquet file and read it into a DataFrame
for file in parquet_files:
    df = pd.read_parquet(file)
    print (f"Read file: '{file}'.  number of rows = {df.shape[0]}")
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
data_df = pd.concat(dfs, ignore_index=True)

print (f"\nTotal number of rows = {data_df.shape[0]}")

Loading data from :  data/granite-docs/output_final/
Number of parquet files to read :  1

Read file: 'data/granite-docs/output_final/Granite_Foundation_Models.parquet'.  number of rows = 216

Total number of rows = 216


In [3]:

## Shape the data

MY_CONFIG.EMBEDDING_LENGTH =  len(data_df.iloc[0]['embeddings'])
print ('embedding length: ', MY_CONFIG.EMBEDDING_LENGTH)

# rename 'embeddings' columns as 'vector' to match default schema
# if 'vector' not in data_df.columns and 'embeddings' in data_df.columns:
#     data_df = data_df.rename( columns= {'embeddings' : 'vector'})
# if 'text' not in data_df.columns and 'contents' in data_df.columns:
#     data_df = data_df.rename( columns= {'contents' : 'text'})

data_df = data_df.rename( columns= {'embeddings' : 'vector', 'contents' : 'text'})

print (data_df.info())
data_df.head(3)

embedding length:  384
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   filename                      216 non-null    object 
 1   num_pages                     216 non-null    int64  
 2   num_tables                    216 non-null    int64  
 3   num_doc_elements              216 non-null    int64  
 4   document_id                   216 non-null    object 
 5   ext                           216 non-null    object 
 6   hash                          216 non-null    object 
 7   size                          216 non-null    int64  
 8   date_acquired                 216 non-null    object 
 9   pdf_convert_time              216 non-null    float64
 10  source_filename               216 non-null    object 
 11  text                          216 non-null    object 
 12  doc_jsonpath                  216 non-nul

,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,...,docq_symbol_to_word_ratio,docq_sentence_count,docq_lorem_ipsum_ratio,docq_curly_bracket_ratio,docq_contain_bad_word,docq_bullet_point_ratio,docq_ellipsis_line_ratio,docq_alphabet_word_ratio,docq_contain_common_en_words,vector
0,Granite%20Foundation%20Models.pdf,20,13,445,efcb95d9-20bb-4997-a306-d2e27d8cccb2,pdf,be4734f0ce53ed18a69ad6cd116a8db3192c7ec7ad2a7c...,455953,2024-07-31T17:17:16.729066,37.919722,...,0.0,2,0.0,0.0,False,0.0,0.0,1.0,True,"[-0.007855909, 0.018679393, 0.042436924, -0.01..."
1,Granite%20Foundation%20Models.pdf,20,13,445,efcb95d9-20bb-4997-a306-d2e27d8cccb2,pdf,be4734f0ce53ed18a69ad6cd116a8db3192c7ec7ad2a7c...,455953,2024-07-31T17:17:16.729066,37.919722,...,0.0,1,0.0,0.0,False,0.0,0.0,1.0,False,"[-0.0035767434, 0.009818679, 0.03441954, -0.00..."
2,Granite%20Foundation%20Models.pdf,20,13,445,efcb95d9-20bb-4997-a306-d2e27d8cccb2,pdf,be4734f0ce53ed18a69ad6cd116a8db3192c7ec7ad2a7c...,455953,2024-07-31T17:17:16.729066,37.919722,...,0.0,6,0.0,0.0,False,0.0,0.0,1.0,True,"[-0.022207903, 0.0050711804, 0.022928528, -0.0..."


## Connect to Vector Database

Milvus can be embedded and easy to use.


In [4]:
from pymilvus import MilvusClient

client = MilvusClient(MY_CONFIG.DB_URI)

print ("✅ Connected to Milvus instance:", MY_CONFIG.DB_URI)

✅ Connected to Milvus instance: ./rag_demo_dataprepkit_1.db


# Create A Collection



In [5]:
# if we already have a collection, clear it first
if client.has_collection(collection_name=MY_CONFIG.COLLECTION_NAME):
    client.drop_collection(collection_name=MY_CONFIG.COLLECTION_NAME)
    print ('✅ Cleared collection :', MY_CONFIG.COLLECTION_NAME)


client.create_collection(
    collection_name=MY_CONFIG.COLLECTION_NAME,
    dimension=MY_CONFIG.EMBEDDING_LENGTH,
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
    auto_id=True
)
print ("✅ Created collection : ", MY_CONFIG.COLLECTION_NAME)


✅ Created collection :  dataprepkit_granite_docs


In [6]:
#data_df.to_dict('records')[:1]

In [7]:
res = client.insert(collection_name=MY_CONFIG.COLLECTION_NAME, data=data_df.to_dict('records'))

print('inserted # rows', res['insert_count'])

client.get_collection_stats(MY_CONFIG.COLLECTION_NAME)

inserted # rows 216


{'row_count': 216}

## Do A Simple Vector Search

We will do this to verify data

In [8]:
from pymilvus import model
import random

# If connection to https://huggingface.co/ failed, uncomment the following path
# import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# embedding_fn = model.DefaultEmbeddingFunction()

## initialize the SentenceTransformerEmbeddingFunction
embedding_fn = model.dense.SentenceTransformerEmbeddingFunction(
    model_name = MY_CONFIG.EMBEDDING_MODEL,
    device='cpu' # this will work on all devices (KIS)
)

## helper function to perform vector search
def  do_vector_search (query):
    # query_vectors = embedding_fn.encode_queries([query])
    query_vectors = embedding_fn([query])

    results = client.search(
        collection_name=MY_CONFIG.COLLECTION_NAME,  # target collection
        data=query_vectors,  # query vectors
        limit=5,  # number of returned entities
        output_fields=["filename", "page_number", "text"],  # specifies fields to be returned
    )
    return results
## ----

def  print_search_results (results):
    # pprint (results)
    print ('num results : ', len(results[0]))

    for i, r in enumerate (results[0]):
        #pprint(r, indent=4)
        print (i+1)
        print ('search score:', r['distance'])
        print ('filename:', r['entity']['filename'])
        print ('page number:', r['entity']['page_number'])
        print ('text:', r['entity']['text'])
        print()

/home/sujee/apps/anaconda3/envs/data-prep-kit-1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
query = "Overview of the Granite Pre-Training Dataset"

results = do_vector_search (query)
print_search_results(results)

num results :  5
1
search score: 0.8775781393051147
filename: Granite%20Foundation%20Models.pdf
page number: 9
text: B. Granite Model Evaluation and Comparison
TABLE II GRANITE.13B GENERAL KNOWLEDGE PERFORMANCE DURING TRAINING

2
search score: 0.8280448913574219
filename: Granite%20Foundation%20Models.pdf
page number: 9
text: B. Granite Model Evaluation and Comparison
For the SocialStigmaQA benchmark, we tested a variety of the Granite, llama-2, and flan-ul2 models. We examine whether the inclusion of specific personal attributes in the prompt leads

3
search score: 0.8216304779052734
filename: Granite%20Foundation%20Models.pdf
page number: 8
text: B. Granite Model Evaluation and Comparison
Fig. 6. Granite.13b General Knowledge Performance during Training.

4
search score: 0.817500114440918
filename: Granite%20Foundation%20Models.pdf
page number: 2
text: C. Organization of Report
The remainder of this report is organized as follows. In Section II, we describe the data sources used in g

In [10]:
query = "Data governance"

results = do_vector_search (query)
print_search_results(results)

num results :  5
1
search score: 0.8625919818878174
filename: Granite%20Foundation%20Models.pdf
page number: 3
text: III. DATA GOVERNANCE
Fig. 2. Summary governance statistics on IBM's curated pre-training dataset at the time of granite.13b.v2's training.

2
search score: 0.8519487380981445
filename: Granite%20Foundation%20Models.pdf
page number: 3
text: III. DATA GOVERNANCE
A. Data clearance and acquisition;
B. Pre-processing; and
C. Tokenization.

3
search score: 0.8367217183113098
filename: Granite%20Foundation%20Models.pdf
page number: 3
text: III. DATA GOVERNANCE
Data governance is organized into the following processes, corresponding to data lifecycle phases prior to model training:

4
search score: 0.8130795955657959
filename: Granite%20Foundation%20Models.pdf
page number: 2
text: III. DATA GOVERNANCE
As IBM is making Granite models available to customers to adapt to their own applications, we have invested heavily in a data governance process that evaluates datasets for governa